In [1]:
%load_ext autoreload
%autoreload 2
import utils as utl

from pathlib import Path
import networkx as nx

import numpy as np
import pandas as pd

# computational set up

In [2]:
# set up environment
path_comp = './2023_03_23_completeness_630_final.csv'
path_con = './2023_03_23_connectivity_630_final.parquet'
path_res = './results/example/'

config = {
    'path_comp'         : path_comp, 
    'path_con'          : path_con, 
    'path_res'          : path_res,     # store results here
    'n_proc'            : 4,            # number of CPUs to use, -1 uses all available CPU
}

In [3]:
# custon neuron names
path_name = './flywire_ids_630.xlsx'

# sheet names of the xls file to include
sheets_pair = [ # pair of neurons (left+right per row)
    'stop',
    'walk',
    'walk_outputs',
    'walk_inputs',
    'SEZ'
    ]

sheets_single = [ # single neurons (one per row)
    'sugar', 
    # 'ovidn', 
    'bitter', 
    'water',
    ]

name2flyid = utl.create_name_dict(path_name, path_comp, sheets_pair, sheets_single)

INFO: Loaded sheets ...
      ... stop
      ... walk
      ... walk_outputs
      ... walk_inputs
      ... SEZ
      ... sugar
      ... bitter
      ... water

159    NaN
196    NaN
203    NaN
Name: Name, dtype: object

INFO: All IDs are unique

Declared 435 names for neurons

ERROR: ID 720575940620306304 for neuron        aDT6_c_l not found. Please provide correct flywire ID
ERROR: ID 720575940611966873 for neuron          aSG1_l not found. Please provide correct flywire ID
ERROR: ID 720575940620267942 for neuron        aSG7_a_l not found. Please provide correct flywire ID
ERROR: ID 720575940617689211 for neuron        aSG7_c_l not found. Please provide correct flywire ID
ERROR: ID 720575940612038257 for neuron        bobber_r not found. Please provide correct flywire ID
ERROR: ID 720575940636683376 for neuron           box_l not found. Please provide correct flywire ID
ERROR: ID 720575940639029592 for neuron brontosaraus_a_l not found. Please provide correct flywire ID
ERROR: ID 7

In [4]:
# define neurons of interest and corresponding group for sorting
# important: groups for input neurons single letters, other multiple letters
d_neu = {
    # neurons we want to watch
    'BPN_SEZ_DN1' : 'bo',
    'BPN_SEZ_DN2' : 'bo',
    'DNg_inh' : 'bo',
    'DNa09' : 'bo',
    'DNg13' : 'bo',
    'mothership' : 'bo',
    'twirl' : 'bo',
    'DNge046' : 'bo',
    'DNg16' : 'bo',
    'DNge050' : 'bo',
    'DNb05' : 'bo',
    'DNgS' : 'mo',
    'MooDNg' : 'mo',
    'MDN-hIN' : 'mo',
    'DNge199' : 'mo',
    'P9-cDN1' : 'po',
    'DNa01' : 'po',
    'P9-0DN1' : 'po',
    'DNa02' : 'po',
    'DNb02' : 'po',
    'DNa06' : 'po',
    'neck motor neuron' : 'po',

    # walk
    'P9' : 'p',
    'MDN_a' : 'm',
    'MDN_b' : 'm',
    'BPN_1a' : 'b',
    'BPN_1b' : 'b',
    'BPN_1c' : 'b',
    'BPN_1d' : 'b',
    'BPN_1e' : 'b',
    'BPN_1f' : 'b',
    'BPN_2a' : 'b',
    'BPN_2b' : 'b',
    'BPN_2c' : 'b',
    'BPN_2d' : 'b',
    'BPN_3a' : 'b',
    'BPN_3b' : 'b',
    'BPN_3c' : 'b',
    'BPN_4a' : 'b',
    'BPN_4b' : 'b',
    'BPN_4c' : 'b',
    'BPN_4d' : 'b',

    # stop
    'FG' : 'F',
    'BB' : 'B',
    'STOP-1_a': 'S',
    'STOP-1_b': 'S',
    'STOP-1_c': 'S',
}

# generate left and right names
s_neu = set()
for n in d_neu.keys():
    nl, nr = f'{n}_l', f'{n}_r'
    s_neu.add(nl) if name2flyid.get(nl) else print(f'couldnt find {nl}')
    s_neu.add(nr) if name2flyid.get(nr) else print(f'couldnt find {nr}')

# put in other hemisphere
name2flyid['DNg_inh_l'] = name2flyid.pop('DNg_inh_r') # need to rerun create namedict if running this cell again
s_neu.remove('DNg_inh_r')
s_neu.add('DNg_inh_l')

couldnt find DNg_inh_l
couldnt find DNg13_r
couldnt find twirl_r
couldnt find DNge199_r
couldnt find P9-0DN1_l
couldnt find P9-0DN1_r
couldnt find neck motor neuron_l
couldnt find BPN_3b_r
couldnt find BPN_3c_r


In [5]:
# load complete connectome into graph (takes a few minutes)
G = utl.get_full_graph(path_comp, path_con)
print(len(G.nodes))

127400


In [6]:
# simulations to consider
ps = [
    'results/coac/walk/P9.parquet',
    'results/coac/walk/MDN.parquet',
    'results/coac/walk/BPN.parquet',

    'results/coac/stop/STOP-1.parquet',
    'results/coac/stop/FG.parquet',
    'results/coac/stop/BB.parquet',

    'results/coac/walk-stop/P9+BB.parquet',
    'results/coac/walk-stop/P9+FG.parquet',
    'results/coac/walk-stop/P9+STOP1.parquet',

    'results/coac/walk-stop/MDN+BB.parquet',
    'results/coac/walk-stop/MDN+FG.parquet',
    'results/coac/walk-stop/MDN+STOP1.parquet',
    
    'results/coac/walk-stop/BPN+BB.parquet',
    'results/coac/walk-stop/BPN+FG.parquet',
    'results/coac/walk-stop/BPN+STOP1.parquet',

    'results/coac/walk-stop-stop/P9+BB+FG.parquet',
    'results/coac/walk-stop-stop/MDN+BB+FG.parquet',
    'results/coac/walk-stop-stop/BPN+BB+FG.parquet',
]


ps_gml = []

for p in ps:
    p = Path(p)

    # write gefx file (for gephi)
    utl.write_graph(G, p, name2flyid=name2flyid, neurons=s_neu)

    # load gefx file
    p_cust = p.parent / (p.with_suffix('').name + '_cust.gexf')
    G_sub = nx.read_gexf(p_cust)
    
    # write gml file (for cytoscape)
    nx.write_gml(G_sub, p_cust.with_suffix('.gml'))
    ps_gml.append(p_cust)

INFO: writing graph file results\coac\walk\P9_cust.gexf
INFO: writing graph file results\coac\walk\MDN_cust.gexf
INFO: writing graph file results\coac\walk\BPN_cust.gexf
INFO: writing graph file results\coac\stop\STOP-1_cust.gexf
INFO: writing graph file results\coac\stop\FG_cust.gexf
INFO: writing graph file results\coac\stop\BB_cust.gexf
INFO: writing graph file results\coac\walk-stop\P9+BB_cust.gexf
INFO: writing graph file results\coac\walk-stop\P9+FG_cust.gexf
INFO: writing graph file results\coac\walk-stop\P9+STOP1_cust.gexf
INFO: writing graph file results\coac\walk-stop\MDN+BB_cust.gexf
INFO: writing graph file results\coac\walk-stop\MDN+FG_cust.gexf
INFO: writing graph file results\coac\walk-stop\MDN+STOP1_cust.gexf
INFO: writing graph file results\coac\walk-stop\BPN+BB_cust.gexf
INFO: writing graph file results\coac\walk-stop\BPN+FG_cust.gexf
INFO: writing graph file results\coac\walk-stop\BPN+STOP1_cust.gexf
INFO: writing graph file results\coac\walk-stop-stop\P9+BB+FG_cust.

In [7]:
# read one of the gml files
G_tot = nx.read_gexf(ps_gml[0])

# add additional attributes we use to help visualization in cytoscape
d = { f'{k}_l': v for k, v in d_neu.items() } | { f'{k}_r': v for k, v in d_neu.items() }
nx.set_node_attributes(G_tot, d, 'group')

m = lambda x: True if len(x) == 1 else False
d = { k: m(v) for k, v in d.items() }
nx.set_node_attributes(G_tot, d, 'input')

d = nx.get_node_attributes(G_sub, 'label')
d = { k : '_'.join(v.split('_')[:-1]) for k, v in d.items() }
nx.set_node_attributes(G_tot, d, 'name'.format(p.name.split('_')[0]).replace('+', ''))

# add the rate attributes based on the gml files
for p in ps_gml:
    G_sub = nx.read_gexf(p, node_type=str)
    n = nx.get_node_attributes(G_sub, 'rate')
    nx.set_node_attributes(G_tot, n, 'rate{}'.format(p.name.split('_')[0]).replace('+', '').replace('-', ''))

# add additional edge attributes we need for cytoscape visualization
s = nx.get_edge_attributes(G_tot, 'sign')
w = nx.get_edge_attributes(G_tot, 'weight')
c = { k: s[k] * w[k] for k in s.keys() }
nx.set_edge_attributes(G_tot, c, 'conn')

# write combinded gml file
nx.write_gml(G_tot, 'walk-stop.gml')